In [ ]:
%reload_kedro

# Loading data

In [ ]:
data_test = catalog.load('scored_test')
model = catalog.load('model')
data_test.head()

# Naive Conformal Predictor Algo

Simple Algo based on the Quantile of the corret class.

Garantee I have a 90% probability of a corret set.

## 1) Get the scores of the correct class

In [ ]:
from numpy import quantile, where

prob_c1 = data_test.prob.values
prob_c0 = 1 - prob_c1
target = data_test.target.values

true_score = [p1 if t == 1 else p0 for p1, p0, t in \
                  zip(prob_c1, prob_c0, target)]

In [ ]:
# check
from numpy import concatenate

concatenate((true_score[: 5], target[: 5], prob_c1[: 5], prob_c0[: 5]), )

## 2) Take the 10% Quantile

In [ ]:
cutoff = quantile(true_score, 0.1)
print("q10 = %4.4f" % cutoff)

## 3) Use this threshold to decide on the decision Set with a garantee of 90%



In [ ]:
decision_set = [('1' if p1 >= cutoff else '', '0' if p0 >= cutoff else '') for p1, p0 in \
                  zip(prob_c1, prob_c0)]

In [ ]:
from numpy import unique

categories, counts = unique(decision_set, return_counts=True, axis=0)
print("categories = %s" % categories)
print("counts = %s" % counts)

In [ ]:
true_score

# Trust Score

In [ ]:
import trustscore

In [ ]:
trust_model = trustscore.TrustScore()

In [ ]:
X = data_test[context.params['features']].values
Y = data_test.target.values
trust_model.fit(X, Y)

In [ ]:
pred = model.predict(X)
trust_model.get_score(X, pred)

In [ ]:
trust_model.get_score(X, pred)

In [ ]:
import trustscore.trustscore_evaluation as trustscore_evaluation

In [ ]:
percentile_levels = [0 + 0.5 * i for i in range(200)]
signal_names = ["Trust Score"]
signals = [trustscore.TrustScore()]
trainer = trustscore_evaluation.run_logistic
trustscore_evaluation.run_precision_recall_experiment_general(X,
                                                            Y,
                                                            n_repeats=10,
                                                            percentile_levels=percentile_levels,
                                                            trainer=trainer,
                                                            signal_names=signal_names,
                                                            signals=signals,
                                                            extra_plot_title='s',
                                                            skip_print=True,
                                                            predict_when_correct=True)
plt.show()